# Tarea 6
Importamos las librerias que se usrán

In [1]:
import pandas as pd
import numpy as np

Primero definimos las funciones que permitirán extraer la data

In [2]:
def extraer_de_csv(archivo_procesar):
    df = pd.read_csv(archivo_procesar)
    return df

In [3]:
def extraer_de_json(archivo_procesar):
    df = pd.read_json(archivo_procesar, lines = True)
    return df

In [4]:
import xml.etree.ElementTree as et
import glob

In [5]:
def extraer_de_xml(archivo_procesar):
    xtree = et.parse(archivo_procesar)
    xroot = xtree.getroot()
    df = pd.DataFrame(columns = ['car_model','year_of_manufacture','price','fuel'])
    for car in xroot:
        car_model = car.find('car_model').text
        year_of_manufacture = int(car.find('year_of_manufacture').text)
        price = float(car.find('price').text)
        fuel = car.find('fuel').text
        df = df.append({'car_model':car_model,'year_of_manufacture':year_of_manufacture,'price':price,'fuel':fuel}, ignore_index = True)
    return df

In [6]:
def extraer():
    extraer_dato = pd.DataFrame(columns = ['car_model','year_of_manufacture','price','fuel'])
    
    for a_csv in glob.glob('./used_car_prices/*.csv'):
        extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
        
    for a_json in glob.glob('./used_car_prices/*.json'):
        extraer_dato = extraer_dato.append(extraer_de_json(a_json), ignore_index = True)
        
    for a_xml in glob.glob('./used_car_prices/*.xml'):
        extraer_dato = extraer_dato.append(extraer_de_xml(a_xml), ignore_index = True)
    return extraer_dato

In [18]:
extraer().head()

C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

,car_model,year_of_manufacture,price,fuel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel


1. Redondear el precio a dos decimales, excluir los que usan Diesel y ordenar por año de manufactura.

In [8]:
def transform(data):
  
    # Redondear el precio a dos decimales
    data['price'] = round(data.price,2)
    
    # Excluir los que usan Diesel    
    data_filtered = data['fuel']!='Diesel'
    data1 = data[data_filtered]
    data2 = data1.set_index('car_model')
    
    # Ordenar por año de manufactura
    data3 = data2.sort_values(by = ['year_of_manufacture'], ignore_index=False)

    return data3

In [19]:
extr = extraer()
transform(extr)

C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

54

In [10]:
def transform2(data):
  
    # Obtener promedio de los precios por modelo, sin importar el año de fabricación
    data1 = data.groupby('car_model').price.sum()
    data2 = data.groupby('car_model').car_model.count()

    data_new = {'average_price': round(data1/data2,2)} 
    data_new2 = pd.concat(data_new, axis = 1) 
    return data_new2 

In [11]:
transform2(extr)

,average_price
car_model,
800,522.39
alto 800,4253.73
alto k10,3835.82
baleno,8731.34
camry,3731.34
ciaz,11152.57
corolla,2238.81
corolla altis,10478.97
dzire,6679.11


In [12]:
import sqlalchemy  

In [13]:
def load(df):
    username = 'root'
    hostname = 'localhost'
    database = 'use_car_prices'
    port     = '3306'
    password = '12345'
    con = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(username, password, 
                                                      hostname, database))
    
    df['id'] = 0
    df.to_sql(con=con, name='precio_modelo', if_exists='append')

In [14]:
from datetime import datetime
now = datetime.now() 
print('Proceso Comienza',now)
extracted_data = extraer()
transformed_data = transform2(extracted_data)
load(transformed_data)
now = datetime.now() 
print('Proceso Finaliza',now)

Proceso Comienza 2022-11-13 13:55:56.692567


C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

Proceso Finaliza 2022-11-13 13:55:56.930365


In [15]:
def load(df):
    username = 'root'
    hostname = 'localhost'
    database = 'use_car_prices'
    port     = '3306'
    password = '12345'
    con = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(username, password, 
                                                      hostname, database))
    
    df['id'] = 0
    df.to_sql(con=con, name='car_tidy', if_exists='append')

In [16]:
from datetime import datetime
now = datetime.now() 
print('Proceso Comienza',now)
extracted_data = extraer()
transformed_data = transform(extracted_data)
load(transformed_data)
now = datetime.now() 
print('Proceso Finaliza',now)

Proceso Comienza 2022-11-13 13:55:56.958629
Proceso Finaliza 2022-11-13 13:55:57.114281


C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  extraer_dato = extraer_dato.append(extraer_de_csv(a_csv), ignore_index = True)
C:\Users\Yhony\AppData\Local\Temp\ipykernel_18236\2867397760.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 